In [1]:
from binn import Network, BINN
import pandas as pd

input_data = pd.read_csv("../data/test_qm.csv")
translation = pd.read_csv("../data/translation.tsv", sep="\t")
pathways = pd.read_csv("../data/pathways.tsv", sep="\t")

network = Network(
    input_data=input_data,
    pathways=pathways,
    mapping=translation,
    source_column="child",
    target_column="parent",
)

binn = BINN(
    network=network,
    n_layers=4,
    dropout=0.2,
    validate=False,
    residual=False,
    device="mps:0",
)

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



BINN is on the device: mps:0


In [2]:
from util_for_examples import fit_data_matrix_to_network_input, generate_data
import torch
from lightning.pytorch import Trainer

design_matrix = pd.read_csv("../data/design_matrix.tsv", sep="\t")

protein_matrix = fit_data_matrix_to_network_input(input_data, features=network.inputs)

X, y = generate_data(protein_matrix, design_matrix=design_matrix)
dataset = torch.utils.data.TensorDataset(
    torch.tensor(X, dtype=torch.float32, device=binn.device),
    torch.tensor(y, dtype=torch.int16, device=binn.device),
)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

trainer = Trainer(max_epochs=3, log_every_n_steps=10)
trainer.fit(binn, dataloader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name   | Type             | Params
--------------------------------------------
0 | layers | Sequential       | 364 K 
1 | loss   | CrossEntropyLoss | 0     
--------------------------------------------
364 K     Trainable params
0         Non-trainable params
364 K     Total params
1.457     Total estimated model params size (MB)
The 'train_dataloader' d

Epoch 2: 100%|██████████| 25/25 [00:00<00:00, 62.91it/s, v_num=78, train_loss=0.791, train_acc=0.533]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 25/25 [00:05<00:00,  4.55it/s, v_num=78, train_loss=0.791, train_acc=0.533]


In [3]:
from binn import BINNExplainer

explainer = BINNExplainer(binn)

In [4]:
test_data = torch.Tensor(X[5:10])
background_data = torch.Tensor(X[0:5])

importance_df = explainer.explain(test_data, background_data)
importance_df.head()

,source,target,source name,target name,value,type,source layer,target layer
0,1,497,A0M8Q6,R-HSA-166663,0.0,0,0,1
1,1,497,A0M8Q6,R-HSA-166663,0.0,1,0,1
2,1,954,A0M8Q6,R-HSA-198933,0.0,0,0,1
3,1,954,A0M8Q6,R-HSA-198933,0.0,1,0,1
4,1,539,A0M8Q6,R-HSA-2029481,0.0,0,0,1


In [5]:
from binn import ImportanceNetwork
import pandas as pd

importance_df = pd.read_csv("../data/importance_df.csv")
IG = ImportanceNetwork(importance_df, norm_method="fan")

In [6]:
IG.plot_complete_sankey(
    multiclass=False, savename="img/test.png", node_cmap="Reds", edge_cmap="Blues"
)

In [7]:
query_node = "R-HSA-68867"

IG.plot_subgraph_sankey(
    query_node, upstream=True, savename="img/test.png", cmap="coolwarm"
)